In [13]:
import pymysql
import pandas as pd
from datetime import date, timedelta

In [18]:
connection = pymysql.connect(host = '119.74.24.181', user = 'htx', password = 'Police123456', database = 'ASTRO')

In [20]:
df = pd.read_sql_query("SELECT * FROM astro.sms", connection)


C:\Users\user\AppData\Local\Temp\ipykernel_20408\245842794.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query("SELECT * FROM astro.sms", connection)


DatabaseError: Execution failed on sql: SELECT * FROM astro.sms
(2013, 'Lost connection to MySQL server during query')
unable to rollback

In [7]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83906 entries, 0 to 83905
Data columns (total 39 columns):
 #   Column                                  Non-Null Count  Dtype 
---  ------                                  --------------  ----- 
 0   report_ number                          83906 non-null  object
 1   initial_report                          83901 non-null  object
 2   classification                          83906 non-null  object
 3   scam_type                               83906 non-null  object
 4   bank                                    83906 non-null  object
 5   account_number                          83906 non-null  object
 6   phone_number_1                          83906 non-null  object
 7   amount_scammed                          83906 non-null  object
 8   amount_transcated                       83906 non-null  object
 9   whatsap_number                          83906 non-null  object
 10  online_market_place                     83906 non-null  object
 11  re

In [8]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
df.tail()

,report_ number,initial_report,classification,scam_type,bank,account_number,phone_number_1,amount_scammed,amount_transcated,whatsap_number,online_market_place,remarks_for_IO,division_assigned,date_assigned,telco,overseas_local,ASC_DO,date_production_order_serve,Time Production Order Sent (HH:MM),Date_bank_account_frozen,Time Confirmed Frozen (HH:MM),Date Bank\nDetails\nReceived,account_holders_name,account_holder's_nric,Contact_Details,latest_balance_seized,coporate_account,referral_date,number_of_days_to_freeze_bank_account,number_of_days_to_receive_bank_details,speed_in_freezing,division,month,MMM,account_or_paynowlah_number,unique_scammedkey,never_appeared_before,report_never_appeared,account_never
83901,D/20230501/4291,ASC-pending SOA,Cheating,Job Scam,UOB,7798633162,,5000.00,$0.00,,,,E,23/3/2023,E/20230322/2057,L-L,Angela Yap,22/3/2023,,22/3/2023,,,HMGLPYCRFVPCNJ,x,,,,3/22/2023,1,None,0:00:00,E,03,Mar,7798633162,,1,0,1
83902,D/20180624/9648,ASC-pending SOA,Cheating,Job Scam,UOB,7693984523,,200.00,$0.00,,,,J,23/3/2023,J/20230322/2067,L-L,Christo Lim,22/3/2023,,23/3/2023,,,SLDXDNXXCTRUVD,x,,,,3/22/2023,2,None,24:00:00,J,03,Mar,7693984523,,1,0,1
83903,G/20100725/3779,ASC-pending SOA,Cheating,Job Scam,UOB,7695289890,,4950.00,$0.00,,,,J,23/3/2023,J/20230322/2067,L-L,Angela Yap,22/3/2023,,23/3/2023,,,CTUUVAWOOHGDTC,x,,,,3/22/2023,2,None,24:00:00,D,03,Mar,7695289890,,1,0,0
83904,F/20180623/4801,ASC-pending SOA,Cheating,Job Scam,UOB,7797948598,94362026,52.00,$0.00,94041108,,,L,23/3/2023,L/20230322/2067,L-L,Angela Yap,23/3/2023,,23/3/2023,,,CCDVQDUVWRAKFV,x,,,,3/23/2023,1,None,0:00:00,F,03,Mar,7797948598,,1,0,1
83905,G/20130222/4107,ASC-pending SOA,Cheating,Job Scam,UOB,7799864630,87312536,1352.00,$0.00,,,,F,23/3/2023,F/20230323/2000,L-L,Angela Yap,23/3/2023,,23/3/2023,,,CIUYSXVSIRKGCD,x,,,,3/23/2023,1,None,0:00:00,F,03,Mar,7799864630,,1,0,1


In [15]:

day_itself = df[df.date_assigned == '23/3/2023']
print(len(day_itself.index))


243


In [6]:
#Number of victims
print(len(df.index))

83906


In [14]:
today = date.today()
dates = []
no_of_cases = []

for i in range(30):
    previous_date = today - timedelta(days = i)
    previous_date = previous_date.strftime("%#d/%#m/%Y")
    dates.append(previous_date)

dates.reverse()

for date in dates:
    df_for_this_date = df[df.date_assigned == date]
    no_of_cases_on_this_date = len(df_for_this_date.index)
    no_of_cases.append(no_of_cases_on_this_date)

print(dates)
print(no_of_cases)

['1/3/2023', '2/3/2023', '3/3/2023', '4/3/2023', '5/3/2023', '6/3/2023', '7/3/2023', '8/3/2023', '9/3/2023', '10/3/2023', '11/3/2023', '12/3/2023', '13/3/2023', '14/3/2023', '15/3/2023', '16/3/2023', '17/3/2023', '18/3/2023', '19/3/2023', '20/3/2023', '21/3/2023', '22/3/2023', '23/3/2023', '24/3/2023', '25/3/2023', '26/3/2023', '27/3/2023', '28/3/2023', '29/3/2023', '30/3/2023']
[251, 233, 267, 227, 97, 129, 411, 182, 179, 284, 222, 130, 143, 332, 276, 202, 211, 188, 125, 137, 433, 194, 243, 0, 0, 0, 0, 0, 0, 0]


In [28]:
#Number of losses
total_loss = df['amount_scammed'].sum()
print(total_loss)

126412.0


In [58]:
#Victims by bank
df.groupby('bank').size().reset_index(name = 'Victims By Banks')

,bank,Victims By Banks
0,DBS/POSB,18
1,OCBC,3


In [55]:
#Types of scams
df.loc[3, 'scam_type'] = 'Job Scam'
df.groupby('scam_type').size()

scam_type
E-commerce Scam     1
Job Scam           16
Loan Scam           3
Love                1
dtype: int64

In [ ]:
#Types of scams
df.loc[3, 'scam_type'] = 'Job Scam'
df.groupby('scam_type').size()